## RÓWNANIA RÓŻNICZKOWE ZWYCZAJNE

In [45]:
import numpy as np
from typing import Callable, Tuple, List

### 1) metoda Eulera:

$$ y' = F(x,y) $$

$$ y(x+h) = y(x) + h F(x,y) $$

In [46]:
def euler(func:Callable[[float,float], float], xStart:float, xStop:float, y0:float, steps:int = None, h:float = None) -> float:
    if h == None:
        h = (xStop - xStart)/ steps
    elif steps == None:
        steps = int((xStop - xStart) / h)
        
    x_value = np.linspace(xStart, xStop, steps + 1)
    y_value, y_value[0] = np.zeros(steps+1), y0
    for i in range(0, steps):
        y_value[i+1] = y_value[i] + h * func(x_value[i], y_value[i])
    return x_value, y_value

### 2) metoda Runge-Kutta II rzędu:

$$ y' = F(x,y) $$

$$ K_0 = h F(x,y)$$

$$ K_1 = h F\left(x+\dfrac{h}{2},y+\dfrac{1}{2}K_0 \right)$$

$$ y(x+h) = y(x) + K_1 $$

In [47]:
def RK_2nd(func:Callable[[float,float], float], xStart:float, xStop:float, y0:float, steps:int = None, h:int = None) -> float:
    if h == None:
        h = (xStop - xStart)/ steps
    elif steps == None:
        steps = int((xStop - x0) / h)

    x_value = np.linspace(xStart, xStop, steps + 1)
    y_value, y_value[0] = np.zeros(steps+1), y0
    for i in range(0, steps):
        x, y = x_value[i], y_value[i]

        k1 = h * func(x, y)
        k2 = h * func(x + h/2, y + k1/2)

        y_value[i+1] = y_value[i] + k2

    return x_value, y_value

### 3) metoda Runge-Kutta IV rzędu:

$$ y' = F(x,y) $$

$$ K_0 = F(x,y)$$

$$ K_1 = F\left(x+\dfrac{h}{2},y+\dfrac{K_0}{2} \right)$$

$$ K_2 = F\left(x+\dfrac{h}{2},y+\dfrac{K_1}{2} \right)$$

$$ K_3 = F\left(x+h,y+K_2 \right)$$

$$ y(x+h) = y(x) + \dfrac{h}{6} (K_0+2K_1+2K_2+K_3) $$

In [48]:
def RK_4th(func:Callable[[float,float], float], xStart:float, xStop:float, y0:float, steps:int = None, h:int = None) -> float:
    if h == None:
        h = (xStop - xStart)/ steps
    elif steps == None:
        steps = int((xStop - xStart) / h)

    x_value = np.linspace(xStart, xStop, steps + 1)
    y_value, y_value[0] = np.zeros(steps+1), y0
    for i in range(0, steps):
        x, y = x_value[i], y_value[i]

        k1 = func(x, y)
        k2 = func(x + h/2, y+h*k1/2)
        k3 = func(x + h/2, y+h*k2/2)
        k4 = func(x + h, y+h*k3)

        y_value[i+1] = y + h/6 *(k1+2*k2+2*k3+k4)

    return x_value, y_value

## RÓWNANIE RÓŻNICZKOWE 2-GO RZĘDU

### 1) metoda Runge-Kutta IV rzędu

In [49]:
def RK_4th_2nd_ODE(func, func_params, y_conditions, tStart, tStop, h = 0.01):
    steps = int((tStop - tStart) / h)
    t_values = np.linspace(tStart, tStop, steps + 1)
    y_values = np.zeros((steps + 1, len(y_conditions)))
    y_values[0] = y_conditions

    for i in range(steps):
        t = t_values[i]
        y = y_values[i]

        k1 = func(t, y, *func_params)
        k2 = func(t + h / 2, y + h * k1 / 2, *func_params)
        k3 = func(t + h / 2, y + h * k2 / 2, *func_params)
        k4 = func(t + h, y + h * k3, *func_params)

        y_values[i + 1] = y + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

    return t_values, y_values[:, 0], y_values[:, 1]

### 2) metoda strzałów - zagadnienia brzegowe

Mamy zagadnienie brzegowe $y(a) = \alpha, \, y(b) = \beta $

Szukamy rozwiązania problemu: $\quad y'' = f(x,y,y'), \quad y(a) = \alpha, \quad y'(a) = u, \,$ dla takiego $\, u=\tilde{u} \, $ aby: $ \quad y(a) = y\left(b,\tilde{u}\right)$

Więc $u$ jest pierwiastkiem wielomianu:
$$ r(u) = y\left(b,u\right) - \beta = y\left(b,u\right) - 1 = 0 $$

In [50]:
def RK_4th_2nd_ODE_v2(func, y_conditions, tStart, tStop, h = 0.01):
    steps = int((tStop - tStart) / h)
    t_values = np.linspace(tStart, tStop, steps + 1)
    y_values = np.zeros((steps + 1, len(y_conditions)))
    y_values[0] = y_conditions

    for i in range(steps):
        t = t_values[i]
        y = y_values[i]

        k1 = func(t, y)
        k2 = func(t + h / 2, y + h * k1 / 2)
        k3 = func(t + h / 2, y + h * k2 / 2)
        k4 = func(t + h, y + h * k3)

        y_values[i + 1] = y + h / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

    return t_values, y_values[:, 0], y_values[:, 1]

In [51]:
def shooting_method(func, y_condA, y_condB, tStart, tStop, max_iter = 100, tol = 10**(-4), h=0.01):
    a, b = tStart, tStop
    for i in range(max_iter):
        u = (a + b) / 2
        y_conditions = [y_condA, u]
        x_values, y0_values, y1_values = RK_4th_2nd_ODE_v2(func, y_conditions, tStart, tStop, h)

        fu = y0_values[-1] - y_condB 

        if np.abs(fu) < tol:
            return x_values, y0_values, y1_values

        y_conditions_a = [y_condA, a]
        _, y0_values_a, _ = RK_4th_2nd_ODE_v2(func, y_conditions_a, tStart, tStop, h)
        fa = y0_values_a[-1] - y_condB

        if fu * fa > 0:
            a = u
        else:
            b = u

### PRZYKŁADOWE ZADANIE 
Znajdź trajektorię piłki rzuconej ukośnie do powierzchni Ziemi

$\textbf(a)$ zaniedbując opory powietrza,

$\textbf(b)$ uwzględniając opory powietrza.

W drugim przypadku załóż, że siła oporu powietrza jest postaci:
$$ F_o = -\dfrac{1}{2} c_{\omega} \rho A |V| \vec{V}$$

gdzie: 

$c_{\omega} \approx 0,35$ - współczynnik oporu powietrza,

$\rho \approx 1,2 \left[\dfrac{kg}{m^3}\right]$ - gęstość powietrza$,

$A \left[m^2\right]$ - pole przekroju poprzecznego piłki,

$ \vec{V} $ - wektor prędkości piłki.

----------------------------------------------------------

#### ROZWIĄZANIE:

A) BEZ OPORÓW POWIETRZA:

- w poziomie OX : $\quad m\vec{a_x} = 0 \iff m \dfrac{dv_x}{dt} = 0 \iff m \dfrac{d^2x}{d^2t} = 0 \quad \implies \dfrac{d^2x}{d^2t} = 0$

Weźmy: $\quad x_0 = x, \quad x_1 = x' \implies x'_0 = x' = x_1. \quad$ 

Zatem:
$$
F(t, x) =
\begin{bmatrix}
x_0' \\
x_1'
\end{bmatrix}
=
\begin{bmatrix}
x_1 \\
0
\end{bmatrix}
$$

$\quad$
- w pione OY : $\quad m\vec{a_y} = -mg \iff m \dfrac{dv_y}{dt} = -mg \iff m \dfrac{d^2y}{d^2t} = -mg \quad \implies \dfrac{d^2y}{d^2t} = -g$

Weźmy: $\quad y_0 = y, \quad y_1 = y' \implies y'_0 = y' = y_1. \quad$ 

Zatem:
$$
F(t, y) =
\begin{bmatrix}
y_0' \\
y_1'
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
-g
\end{bmatrix}
$$

In [52]:
def Fx_noAir(t, x_conditions):
    x0, x1 = x_conditions
    F = np.zeros(2)
    F[0] = x1
    F[1] = 0
    return F

def Fy_noAir(t, y_conditions):
    g = 9.81
    y0, y1 = y_conditions
    F = np.zeros(2)
    F[0] = y1
    F[1] = -g
    return F

In [53]:
def simulate_noAir(v0, angle, tStart, tStop, h = 0.01):
    angle_rad = np.radians(angle)
    vx0 = v0 * np.cos(angle_rad)
    vy0 = v0 * np.sin(angle_rad)

    t_x, x_values, _ = RK_4th_2nd_ODE_v2(Fx_noAir, [0, vx0], tStart, tStop, h)
    t_y, y_values, _ = RK_4th_2nd_ODE_v2(Fy_noAir, [0, vy0], tStart, tStop, h)

    for i, yi in enumerate(y_values):
        if yi<0:
            return t_x[:i], x_values[:i], y_values[:i]

    return t_x, x_values, y_values

B) Z OPORAMI POWIETRZA:

- w poziomie OX : $\quad m\vec{a_x} = -F_o \iff m \dfrac{dv_x}{dt} = -F_{o_x} \iff m \dfrac{d^2x}{d^2t} = -F_{o_x} \quad \implies \dfrac{d^2x}{d^2t} = -\dfrac{1}{m}F_{o_x}$

Weźmy: $\quad x_0 = x, \quad x_1 = x' \implies x'_0 = x' = x_1. \quad$ 

Zatem:
$$
F(t, x) =
\begin{bmatrix}
x_0' \\
x_1'
\end{bmatrix}
=
\begin{bmatrix}
x_1 \\
-\dfrac{1}{m}F_{o_x}
\end{bmatrix}
$$

$\quad$

- w pione OY : $\quad m\vec{a_y} = -mg - F_{o_y} \iff m \dfrac{dv_y}{dt} = -mg - F_{o_y} \iff m \dfrac{d^2y}{d^2t} = -mg - F_{o_y} \quad \implies \dfrac{d^2y}{d^2t} = -g-\dfrac{1}{m}F_{o_y}$

Weźmy: $\quad y_0 = y, \quad y_1 = y' \implies y'_0 = y' = y_1. \quad$ 

Zatem:
$$
F(t, y) =
\begin{bmatrix}
y_0' \\
y_1'
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
\\
-g-\dfrac{1}{m} F_{o_y}
\end{bmatrix}
$$

In [54]:
def air_resistance(cw, rho, A, vx, vy):
    v = np.sqrt(vx**2 + vy**2)
    return -0.5 * cw * rho * A * v

def Fx_withAir(t, x_conditions, cw, rho, A, vx, vy, m):
    x0, x1 = x_conditions
    F = np.zeros(2)
    F[0] = x1
    F[1] = air_resistance(cw, rho, A, vx, vy) * vx / m
    return F

def Fy_withAir(t, y_conditions, cw, rho, A, vx, vy, m):
    y0, y1 = y_conditions
    F = np.zeros(2)
    F[0] = y1
    F[1] = -g + air_resistance(cw, rho, A, vx, vy) * vy / m
    return F

In [55]:
def simulate_withAir(v0, angle, func_params, tStart, tStop, h=0.01):
    angle_rad = np.radians(angle)
    vx0 = v0 * np.cos(angle_rad)
    vy0 = v0 * np.sin(angle_rad)

    cw, rho, A, m = func_params

    t_x, x_values, _ = RK_4th_2nd_ODE(Fx_withAir, (cw, rho, A, vx0, vy0, m), [0, vx0], tStart, tStop, h)
    _, y_values, _ = RK_4th_2nd_ODE(Fy_withAir, (cw, rho, A, vx0, vy0, m), [0, vy0], tStart, tStop, h)

    for i, y in enumerate(y_values):
        if y < 0:
            return t_x[:i], x_values[:i], y_values[:i]

    return t_x, x_values, y_values

In [56]:
tStart = 0
tStop = 10
rho = 1.2
cw = 0.35
g = 9.81

In [57]:
v01 = 10
angle1 = 45  
m1 = 0.145  
A1 = 0.0042

In [58]:
t1_noAir, x1_noAir, y1_noAir = simulate_noAir(v01, angle1, tStart, tStop)

In [59]:
t1_withAir, x1_withAir, y1_withAir = simulate_withAir(v01, angle1, (cw,rho,A1,m1), tStart, tStop)